## Basic functions

In [1]:
import tensorflow as tf  # tf 2.x
import matplotlib.pyplot as plt
import math
import numpy as np
from numpy.random import choice

try:
    import QGOpt as qgo
except ImportError:
    !pip install git+https://github.com/LuchnikovI/QGOpt
    import QGOpt as qgo

In [2]:
def matr_Ry(angle):
    return np.array([[math.cos(angle/2), -math.sin(angle/2)],
              [+math.sin(angle/2), math.cos(angle/2)]])

def matr_Rx(angle):
    return np.array([[math.cos(angle/2), -1j * math.sin(angle/2)],
              [-1j * math.sin(angle/2), math.cos(angle/2)]])

In [3]:
def kron(A, B):
    """
    Returns Kronecker product of two square matrices.

    Args:
        A: complex valued tf tensor of shape (dim1, dim1)
        B: complex valued tf tensor of shape (dim2, dim2)

    Returns:
        complex valued tf tensor of shape (dim1 * dim2, dim1 * dim2),
        kronecker product of two matrices
    """

    dim1 = A.shape[-1]
    dim2 = B.shape[-1]
    AB = tf.transpose(tf.tensordot(A, B, axes=0), (0, 2, 1, 3))
    return tf.reshape(AB, (dim1 * dim2, dim1 * dim2))

A convenient way to present circuits is to create an array which will encode the sequence of gates into the sequence of ints. This is especially useful in the 1-qubit case, when you simply have several martices which act on the qubit one-by-one. 

The right way is to work with sets of gates rather than circuits. That means we need to define one pure set of gates and their noised versions for each qubit. 'big_unitary_bank' is going to be our workspace, one which we will make our variable u and change with each iteration. As for now, 'big_unitary_bank' has only one row - because we have only one qubit. The array 'true_unitary_bank' holds answers to our task; we hope that with iterations we will get 'big_unitary_bank' as close to 'true_unitary_bank' as possible.

In [4]:
set_length = 4
# 0 - Rx(pi/2)
# 1 - Ry(pi/2)
# 2 - Rx(pi/4)
# 3 - Rx(pi/8)

In [5]:
big_unitary_bank = []
pure_unitary_set = np.empty(shape=(set_length, 2, 2), dtype = 'complex_')
pure_unitary_set[0] = matr_Rx(math.pi/2)
pure_unitary_set[1] = matr_Ry(math.pi/2)
pure_unitary_set[2] = matr_Rx(math.pi/4)
pure_unitary_set[3] = matr_Rx(math.pi/8)

qubit_noised_set = pure_unitary_set.copy()
big_unitary_bank.append(qubit_noised_set)

In [6]:
true_unitary_bank = []
true_noised_set = pure_unitary_set.copy()

#NOISING OF THE SET

true_noised_set[2] = np.diag((1, 1))

#NOISING DONE

true_unitary_bank.append(true_noised_set)

For now, we imply a restriction: even though we have some noise, our noisy gates are still considered unitary. In the future versions of this code, we need to work with Choi matrices that represent arbitrary quantum channels instead.

In [7]:
m = qgo.manifolds.StiefelManifold()

Since we have only one qubit, the circuit can be written as a single row of 2x2 matrices. The '1d' in function indicates that these functions build only one row of sinqle-qubit gates.

In [8]:
def build_1d_circuit_from_map(array, qubit):
    a = np.empty(shape=(len(array), 2, 2), dtype = 'complex_')
    for i in np.arange(len(array)):
        a[i] = big_unitary_bank[qubit][array[i]]       
    return a

def build_1d_true_circuit_from_map(array, qubit):
    a = np.empty(shape=(len(array), 2, 2), dtype = 'complex_')
    for i in np.arange(len(array)):
        a[i] = true_unitary_bank[qubit][array[i]]       
    return a

These functions take array of ints and convert them to an array of 2x2 matrices represenging gates. The first function will be called at each optimization iteration and will re-build the circuit from new version of 'big_unitary_bank'. The second one is not needed in real program; it's called once to generate samples the user should provide in normal circumstances.

## Circuit maps initialization

The circuit structure has changed. Right now it has only three indices:
1) Number of the gate in the circuit (in multi-qubit version this has to be marked with two numbers when the circuits will have more than one line) <br>
2) & 3) are indices for locating the matrix element in the unitary gate.

We're not working anymore with one HUGE array that stores all the circuits: we have circuits_map for this now.
'circuits_map' has two indices:
1) circuit number <br>
2) Number of the gate in the circuit <br>
'circuits_map' stores ints instead of 2x2 matrices, but this int is actually a reference to 2x2 matrix in 'big_unitary_bank'

In [9]:
circuits_map = []
#circuits_map.append([0, 1])
circuits_map.append([0, 1, 2])
#circuits_map.append([0, 1, 0])
#circuits_map.append([2])

In [10]:
ideal_circuits_representation = []
for circ_number in np.arange(len(circuits_map)):
    ideal_circuits_representation.append(build_1d_circuit_from_map(circuits_map[circ_number], 0)) 
    #we use the fact that big_unitary_bank is not noised yet so we can use function build_1d_circuit_from_map
    #ideal_circuits_representation stores ideal versions of our gates in the 2x2 matrix form

Instead of having three copies of circuits mega-array, we're working with one circuit at a time, which we'll rebuild at each optimization step using the 'build_1d_circuit_from_map' function.

Still, we will have one ideal mega-array stored not in terms of ints but in terms of 2x2 matrices so it will be easier and faster to compute matrix norm

## Sample initialization

This function should emulate 1000 circuits applied to a state |0>. In single-qubit version it's easy to do so because we can just multiply every matrix and then look at the top row of our 2x2 matrix. These two numbers will define the probability of yielding the state |0> or |1> as the result.

In [11]:
def generate_samples(circs_map):
    samples = []
    for it in np.arange(len(circs_map)): #we iterate over circuits
        sample = []
        circ_map = circs_map[it]
        uc = build_1d_true_circuit_from_map(circ_map, 0) #0 here means we take the gates accosiated with 1st qubit
        big_u = np.diag((1, 1))
        for j in np.arange(len(uc)):
            big_u = big_u @ uc[j]
        probs = abs(big_u[0])/sum(abs(big_u[0]))
        for _ in np.arange(1000):
            sample.append(choice((0, 1), 1, p=probs))
        samples.append(sample)
    
    return samples

In [12]:
samples = generate_samples(circuits_map)

## Working with the optimizer

Now we prepare the variables as written in guide and try to input this in our optimizer.

In [13]:
u = qgo.manifolds.complex_to_real(big_unitary_bank)

In [14]:
u = tf.Variable(u)

In [15]:
lr = 0.02 #optimization step size
opt = qgo.optimizers.RAdam(m, lr)

In our model we need to get logarithm of probability that we got the correct gates. Since this is calculated via multiplication of probabilities to obtain each sample, we can just sum the ln(probability[i-th sample]) for i = 0..999.
The probability itself is easy to check since our final operator is a product of 2x2 matrices multilpication. Then we just look at <sample_bit| (u0 @ u1 @ ... @ uN) |0> which is the same as abs of the matrix element 

In [16]:
def get_log_probability(row_of_unitaries, sample_num, circ_num):
    bit_string = samples[circ_num][sample_num] #it = circuit number, i - number of sample
    indexx = bit_string[0] #yes this is retarded the 'bit_string' is an array of 1 element 
    #but when the bit string becomes string instead of a single nubmer this will become logical
    big_u = np.diag((1, 1))
    for j in np.arange(len(row_of_unitaries)):
        big_u = big_u @ row_of_unitaries[j]
    #Because our qubit is affected only by single-qubit operations, we can calculate each qubit independently
    #That means the probability^2 is just gonna be |big_u[bit_string][0]|^2
    #print(big_u[bit_string[0]][0])
    #print (big_u)
    if (abs(big_u[indexx][0]) == 0):
        return -100
    else:
        return math.log(abs(big_u[indexx][0]))

In [18]:
# this list will be filled by value of
# error per iteration
err_vs_iter = []

#sigma is a parameter for gaussian regularization
sigma = 1 

# optimization loop
for _ in range(100):
    with tf.GradientTape() as tape:
        # turning u back into its complex representation
        big_unitary_bank = qgo.manifolds.real_to_complex(u) 
        L_reg = 0
        L_1 = 0
        
        for it in np.arange(len(circuits_map)): #we iterate with variable 'it' over CIRCUITS 
            circuit = build_1d_circuit_from_map(circuits_map[it], 0)
            ideal_circuit = ideal_circuits_representation[it]
            # Regularization (1st term)
            for i in np.arange(len(circuit)): #we iterate with variable 'i' over GATES in a circuit 
                L_reg += tf.linalg.norm(circuit[i] - ideal_circuit[i]) / 2
                #print('add Lreg', i, it)
        
            # loss function (2nd term)
            for i in np.arange(len(samples[it])): #we iterate with variable 'i' over 1000 SAMPLES related to a circuit 'it' 
                L_1 += 2 * get_log_probability(circuit, i, it)
                #print('add L1', i, it)
        
        #print('Reg =', tf.math.real(L_reg / sigma), 'Func =', L_1, '\n')
        L = tf.math.real(L_reg / sigma - L_1) #we wanted to MAXIMIZE (log_prob - reg) => we MINIMIZE (reg - log_prob)
        print(L)
        
    # filling list with history of error
    err_vs_iter.append(L)
    # gradient from tape
    grad = tape.gradient(L, u)
    print('grad = ', grad)
    # optimization step
    opt.apply_gradients(zip([grad], [u]))

tf.Tensor(1055.585495406282, shape=(), dtype=float64)
grad =  None


ValueError: No gradients provided for any variable: ['Variable:0'].

In [ ]:
plt.plot(err_vs_iter)
plt.yscale('log')
plt.xlabel('iter')
plt.ylabel('err')

In [ ]:
big_unitary_bank = qgo.manifolds.real_to_complex(u)

In [ ]:
big_unitary_bank

In [ ]:
true_unitary_bank

In [ ]:
pure_unitary_set